In [ ]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
# from google.cloud import bigquery
import json

client = MongoClient('localhost', 27017)

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
# bq_client = bigquery.Client()

db = client.worldcup

load_dotenv()

tweepy_client = tweepy.Client(os.getenv('BEARER_TOKEN'), wait_on_rate_limit=True)

In [ ]:
query = '#ARG OR #HRV -is:retweet'

paginator = tweepy.Paginator(tweepy_client.search_recent_tweets, 
                        query=query,
                        tweet_fields = ['author_id', 'created_at', 'source', 'lang', 'geo', 'public_metrics', 'entities', 'context_annotations', 'attachments'], 
                        media_fields = ['preview_image_url', 'media_key', 'type', 'url'],
                        exclude=['retweets', 'replies'],
                        expansions = ['author_id', 'attachments.media_keys'], 
                        start_time = '2022-12-13T00:00:00.000Z',
                        end_time = '2022-12-14T00:00:00.000Z',
                        max_results = 100
                        )

for page in paginator:
    
    users = {u["id"]: u for u in page.includes['users']}
    media = {m["media_key"]: m for m in page.includes['media']}

    for tweet in page.data:

        if users[tweet.author_id]:
            user = users[tweet.author_id]

        # if media:
        #     attachments = tweet.data['attachments']
        #     media_keys = attachments['media_keys']
        #     media_info = {
        #         "media_key": media_keys,
        #         "media_type": media[media_keys[0]].type,
        #         "preview_image_url": media[media_keys[0]].preview_image_url
        #     }
        # else:
        #     media_info = ""
        
        tweet_content = {
            "tweet_id": tweet.id,
            "author": user,
            "created_at": tweet.created_at,
            "source": tweet.source,
            "lang": tweet.lang,
            "geo": tweet.geo,
            "public_metrics": tweet.public_metrics,
            "entites": tweet.entities,
            "text": tweet.text,
            # "media": media_info
        }
        
        db.testarghrv.insert_one(tweet_content)

# for tweet in tweets.flatten():
#     try:
#         print("1. ",tweet.data['entities'])
#         print("2. ",tweet.includes['name'])
#         for key, value in tweet.data.items():
#             print (key, value)
#         # db.a.insert_one(tweet)
#         break
#     except ValueError as e:
#         print(e)

In [ ]:
tweets_db = db.semiFinalArgHrv.find()

In [ ]:
import datetime
for tweet in tweets_db:
    content = {
        "tweet_id": tweet['tweet_id'],
        "created_at": tweet['created_at']
    }

    print(content)
    break

In [ ]:
for tweet in tweets_db:
    print(tweet)
    break

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
raw = 'RT @FMPinilla: #Messi #ARG #Argentina #Messi𓃵 #Qatar2022 https://t.co/6UnavQ9zBn'
token = nltk.word_tokenize(raw)

from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('english'))
punct_set = set(string.punctuation)
filtered_tokens = [w for w in token
                    if not w.lower() in stop_words and 
                     not punct_set.issuperset(set(w))]

In [ ]:
print(filtered_tokens)

In [ ]:
d = {"a": {"b":1}}

print(type(d))
print(d["a"].get("c"))
print(type(d["a"].get("c")))


In [ ]:
import datetime
for tweet in tweets_db:
    hashtags = []
    if tweet['entities'].get('hashtags'):
        for hashtag in tweet['entities']['hashtags']:
            hashtags.append(hashtag['tag'])
    print(hashtags)

    annotations = []
    annotations_person = []
    if tweet['entities'].get('annotations'):
        for anno in tweet['entities']['annotations']:
            if anno['type'] == "Person":
                annotations_person.append(anno['normalized_text'])
            else:
                annotations.append(f'({anno["type"]}) {anno["normalized_text"]}')
    print(annotations)
    print(annotations_person)

    con = {
        "hashtags": ", ".join(hashtags),
        "annotations": ", ".join(annotations),
        "annotations_person": ", ".join(annotations_person)
    }
    print(con)
    # break
    # print(tweet.text)
    # print(tweet['created_at'].date())
    # print(tweet['created_at'].time())
    # print(tweet['tweet_id'])
    # print(tweet['author']['id'])

    break


In [ ]:
{'_id': ObjectId('639b1b51c2728b12d0639cd1'), 
 'tweet_id': 1602815590644830210, 
 'author': {'id': 138911752, 
            'name': 'ricardo leon crespo', 
            'username': 'tatuleonc'}, 
 'created_at': datetime.datetime(2022, 12, 13, 23, 59, 50), 
 'source': 'Twitter for Android', 
 'lang': 'es', 
 'geo': None, 
 'public_metrics': {'retweet_count': 154, 
                    'reply_count': 0, 
                    'like_count': 0, 
                    'quote_count': 0}, 
 'entites': {'mentions': [{'start': 3, 
                           'end': 12, 
                           'username': 'Jwillocv', 
                           'id': '1552259230547296256'}], 
             'annotations': [{'start': 59, 
                              'end': 63, 
                              'probability': 0.9821, 
                              'type': 'Person', 
                              'normalized_text': 'Messi'}, 
                             {'start': 67, 
                              'end': 72, 
                              'probability': 0.968, 
                              'type': 'Person', 
                              'normalized_text': 'Armani'}, 
                            {'start': 79, 
                             'end': 81, 
                             'probability': 0.9358, 
                             'type': 'Organization', 
                             'normalized_text': 'ARG'}, 
                            {'start': 84, 'end': 97, 
                             'probability': 0.5483, 
                             'type': 'Organization', 
                             'normalized_text': 'VamosArgentina'}, 
                            {'start': 100, 'end': 111, 
                             'probability': 0.8338, 
                             'type': 'Other', 
                             'normalized_text': 'FIFAWorldCup'}, 
                            {'start': 114, 'end': 122, 
                             'probability': 0.8587, 
                             'type': 'Other', 
                             'normalized_text': 'Qatar2022'}], 
             'hashtags': [{'start': 78, 'end': 82, 'tag': 'ARG'}, 
                          {'start': 83, 'end': 98, 'tag': 'VamosArgentina'}, 
                          {'start': 99, 'end': 112, 'tag': 'FIFAWorldCup'}, 
                          {'start': 113, 'end': 123, 'tag': 'Qatar2022'}]}, 
 'text': 'RT @Jwillocv: Hermosooooooooooooooos 😍😍😍😍\n\nEl abrazo entre Messi y Armani 💙🤍💙\n#ARG #VamosArgentina #FIFAWorldCup #Qatar2022 https://t.co/Kp…'}

In [ ]:
import datetime
a = datetime.datetime(2022, 12, 13, 23, 59, 57)
print (a.date())

In [ ]:
jpn_vs_hrv = bq_client.get_table('tecky-capstone-project.worldcup.jpn_hrv')

x = db.jh.find()
for row in x:
    row['_id'] = str(row['_id'])
    row['Date_Created'] = str(row['Date_Created'])
    bq_client.insert_rows_json(jpn_vs_hrv,[row])